### Dicionarios das variaveis

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pingouin as pg
from pingouin import pairwise_corr
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pingouin as pg
import random 
import scipy.stats
from scipy import stats
import janitor

In [5]:
base_PAG_HEI = pd.read_csv('PAG_HEI.csv')
base_PHQ = pd.read_csv('DEMO_PHQ.csv')

### 1. Agrupamento dos bancos de dados

In [18]:
#VERIFICAÇÃO DOS VALORES DE CADA BASE E COLUNAS
#base_PAG_HEI.head()
#base_PHQ.head()

base_PAG_HEI.columns
base_PHQ.columns

Index(['SEQN', 'DPQ010', 'DPQ020', 'DPQ030', 'DPQ040', 'DPQ050', 'DPQ060',
       'DPQ070', 'DPQ080', 'DPQ090', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1',
       'DMDEDUC', 'INDFMINC'],
      dtype='object')

In [14]:
#VERIFICANDO TAMANHO DAS BASES
base_PAG_HEI.shape
base_PHQ.shape

(5334, 15)

In [21]:
#JOIN DAS COLUNAS POR MÉTODO OUTER JOIN, PARA PEGAR AS COLUNAS ÚNICAS DAS DUAS BASES
base = base_PHQ.merge(base_PAG_HEI, on='SEQN', how='outer')
base.to_csv('base_agrupada.csv')

### 2. Inspeção da base de dados agrupada

#### 2.1 Verificação e alteração dos tipos de dados nas colunas


In [65]:
#OBTENDO OS TIPOS DAS COLUNAS
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9424 entries, 0 to 9423
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SEQN                      9424 non-null   int64  
 1   DPQ010                    4836 non-null   float64
 2   DPQ020                    4836 non-null   float64
 3   DPQ030                    4836 non-null   float64
 4   DPQ040                    4836 non-null   float64
 5   DPQ050                    4836 non-null   float64
 6   DPQ060                    4836 non-null   float64
 7   DPQ070                    4836 non-null   float64
 8   DPQ080                    4836 non-null   float64
 9   DPQ090                    4836 non-null   float64
 10  RIAGENDR                  5334 non-null   float64
 11  RIDAGEYR                  5334 non-null   float64
 12  RIDRETH1                  5334 non-null   float64
 13  DMDEDUC                   5334 non-null   float64
 14  INDFMINC

In [66]:
'''AS COLUNAS DO PHQ TEM O VALOR EM FLOAT E SERÃO SOMADAS PARA OBTENÇÃO DO SCORE, NO ENTANTO, OS VALORES "7" E "9" SÃO REFERENTES ÀS PESSOAS QUE NÃO RESPONDERAM O QUESTIONÁRIO E, POR ISSO, DEVEM SER DESCONSIDERADOS. POR HORA, VOU VERIFICAR O NÚMERO DE SETES E NOVES EM CADA UMA DESSAS COLUNAS E ALTERAREI OS VALORES PARA 0
NÃO HÁ NENHUM NÚMERO "7" E OS "9" SÃO POUCOS NÚMEROS, ENTÃO A ALTERAÇÃO NÃO DEVE MODIFICAR MUITO A LEITURA DA BASE'''
#base.info()

for column in base.iloc[:, 1:10]:
#    print(base[column].value_counts())
    base[column].replace(9.0, 0.0, inplace=True)

In [78]:
#ALGUMAS VARIÁVEIS CATEGÓRICAS ESTÃO COMO TIPO FLOAT. ALTERO PARA 'CATEGORY' PARA FACILITAR A ANÁLISE
for column in ['RIAGENDR', 'RIDRETH1', 'DMDEDUC', 'INDFMINC', 'ADHERENCE']:
#    print(base[column].value_counts())
    base[column] = base[column].astype('category')

base.info()

2.0    2773
1.0    2561
Name: RIAGENDR, dtype: int64
3.0    2516
4.0    1300
1.0    1133
5.0     221
2.0     164
Name: RIDRETH1, dtype: int64
4.0    1498
3.0    1335
5.0     939
2.0     936
1.0     618
7.0       4
9.0       4
Name: DMDEDUC, dtype: int64
11.0    1044
6.0      678
7.0      535
8.0      485
3.0      440
5.0      426
4.0      412
9.0      297
2.0      281
10.0     278
1.0      190
12.0      78
99.0      77
77.0      36
13.0      30
Name: INDFMINC, dtype: int64
3.0    3295
1.0    2761
2.0    1094
Name: ADHERENCE, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9424 entries, 0 to 9423
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   SEQN                      9424 non-null   int64   
 1   DPQ010                    4836 non-null   float64 
 2   DPQ020                    4836 non-null   float64 
 3   DPQ030                    4836 non-null   float64 
 4   DPQ040  

In [95]:
#FALTA, AINDA, CHECAR SE NÃO EXISTEM VALORES ESTRANHOS NOS SCORES DO HEI

#erros = base[base['HEI2015C1_TOTALVEG'].between(0,5)]

for item in base.loc[:, 'HEI2015C1_TOTALVEG': 'HEI2015C13_ADDSUG']:
    if base.loc[base[item]] < 0:
        print('Problema')


       SEQN  DPQ010  DPQ020  DPQ030  DPQ040  DPQ050  DPQ060  DPQ070  DPQ080  \
1     31131     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2     31132     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3     31134     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     31139     0.0     0.0     0.0     0.0     3.0     1.0     0.0     0.0   
5     31143     0.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
...     ...     ...     ...     ...     ...     ...     ...     ...     ...   
9418  41448     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9420  41456     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9421  41467     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9422  41471     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9423  41474     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

      DPQ090  ... HEI2015C5_WHOLEGRAIN  HEI2015C6_T

#### 2.2 Tratamento de valores nulos

In [ ]:
#ENCONTRANDO OS VALORES NULOS DA BASE